this report generates all plots for ablation study.


there are four sets of data to read

1. original recurrent models, 2L, 16/32 ch, average depth as well as performance.
2. multipath models, 2L, 16/32 ch, average depth as well as performance.
3. ablated multipath models, onlyDX, 2L, 16/32 ch, average depth as well as performance.
4. ablated multipath models, leDX/geDX, 2L, 16/32 ch, average depth as well as performance.

I should denote them with different colors.


I should plot data at two levels.

1. overall level
2. per read out type. this is to show that 8K data does not work well with "single path does not work" hypothesis


storage format.

a big dataframe

* `data_source`: 'original', 'multipath', 'onlyDX'. 'leDXgeDX'
* `aggregate_level`: '', 'readout_type=inst-avg', 'readout_type=inst-last', 'readout_type=cm-avg', 'readout_type=cm-last'
    * '' means everything
    * more general, it's `/` joined by `k=v`
* `rcnn_bl_cls`: 2,3,4,5,6,7
    * no feed-forward models involved
* `n`: how many raw models are involved to get every number that follows
* `depth_scalar_mean`: a list of depths.
    * for `leDX/geDX`, this will be a proper list, with 2*`rcnn_bl_cls` numbers, following the order in plots in `/results_processed/tang_refactored/maskcnn_polished_with_rcnn_k_bl/20201218_plot.ipynb`
    * for others, this will be a list with a single number.
* `depth_scalar_sem`: similar to above, but sem. calculated with `sem(ddof=0)`.
* `cc2_normed_avg_mean`: similar to `depth_scalar_mean`
* `cc2_normed_avg_sem`: similar to `depth_scalar_sem`


Note that, for every combination of (`data_source`, `aggregate_level`, `rcnn_bl_cls`), the `n` should be the same and only and all models that are available in `multipath` should be analzyed. This is because `multipath` models have the most demanding requirement on memory so there are fewest of them.

In [1]:
from os.path import join, dirname, exists
from os import makedirs
from thesis_v2 import dir_dict

In [2]:
import pandas as pd
import numpy as np

In [3]:
from thesis_v2.analysis.ablation_study import collect_all_data

In [4]:
def load_original_perf():
    # this does not contain 48, 64 channel results, though.
    # (I forgot to train these models in ensemble mode)
    f_main_result_perf = join(dir_dict['analyses'], 'yuanyuan_8k_a_3day_refactored', '20200725+20200801+20200801_2', 'main_result.pkl')
    df_main_result_perf = pd.read_pickle(f_main_result_perf).sort_index()
    
    return df_main_result_perf

In [5]:
def load_original_depth():
    # this does not contain 48, 64 channel results, though.
    # (I forgot to train these models in ensemble mode)
    f_main_result = join(dir_dict['analyses'], 'yuanyuan_8k_a_3day_refactored', '20200725+20200801+20200801_2', 'source_analysis.pkl')
    df_main_result = pd.read_pickle(f_main_result).sort_index()
    return df_main_result

In [6]:
def load_multipath_perf():
    ret = pd.concat(
        [
            pd.read_pickle(join(dir_dict['analyses'], 'yuanyuan_8k_a_3day_refactored', '20201118', 'main_result_separatebn.pkl')),
            pd.read_pickle(join(dir_dict['analyses'], 'yuanyuan_8k_a_3day_refactored', '20201114', 'main_result_separatebn.pkl')),
        ], axis=0
    )
    return ret

def load_multipath_depth():
    ret = pd.read_pickle(join(dir_dict['analyses'], 'yuanyuan_8k_a_3day_refactored', '20201114+20201118_separatebn', 'source_analysis.pkl'))
    return ret

In [7]:
def load_onlyDX_perf():
    ret = pd.concat(
        [
            pd.read_pickle(join(dir_dict['analyses'], 'yuanyuan_8k_a_3day_refactored', '20201221', 'main.pkl')),
        ], axis=0
    )
    return ret

In [8]:
def load_onlyDX_depth():
    ret = pd.read_pickle(join(dir_dict['analyses'], 'yuanyuan_8k_a_3day_refactored', '20201221', 'source_analysis.pkl'))
    return ret

In [9]:
def load_leDXgeDX_perf():
    ret = pd.concat(
        [
            pd.read_pickle(join(dir_dict['analyses'], 'yuanyuan_8k_a_3day_refactored', '20201205+20201205_2', 'main.pkl')),
            pd.read_pickle(join(dir_dict['analyses'], 'yuanyuan_8k_a_3day_refactored', '20201213+20201213_2', 'main.pkl')),    
        ], axis=0
    )
    return ret

In [10]:
def load_leDXgeDX_depth():
    f_main_result_src = join(dir_dict['analyses'], 'yuanyuan_8k_a_3day_refactored', '20201205+20201205_2', 'source_analysis.pkl')
    f_main_result_2_src = join(dir_dict['analyses'], 'yuanyuan_8k_a_3day_refactored', '20201213+20201213_2', 'source_analysis.pkl')
    
    df_main_result_src = pd.read_pickle(f_main_result_src)
    df_main_result_2_src = pd.read_pickle(f_main_result_2_src)
    
    df_main_result_src = pd.concat([df_main_result_src, df_main_result_2_src], axis=0).sort_index()
    assert df_main_result_src.index.is_unique
    
    return df_main_result_src

In [11]:
f_main_result = join(dir_dict['analyses'], 'yuanyuan_8k_a_3day_refactored', 'ablation_2L_16,32ch_5120.pkl')
makedirs(dirname(f_main_result), exist_ok=True)

if not exists(f_main_result):
    df_main_result = collect_all_data(
    data_loader_dict = {
        'multipath': {
            'perf': load_multipath_perf,
            'depth': load_multipath_depth,
        },
        'original': {
            'perf': load_original_perf,
            'depth': load_original_depth,
        },
        'onlyDX': {
            'perf': load_onlyDX_perf,
            'depth': load_onlyDX_depth,
        },
        'leDXgeDX': {
            'perf': load_leDXgeDX_perf,
            'depth': load_leDXgeDX_depth,
        }
    },
        train_keep=5120,
    )
    df_main_result.to_pickle(f_main_result)
    del df_main_result
df_main_result = pd.read_pickle(f_main_result)

(736,)


In [12]:
df_main_result

depth_mean  \
data_source aggregate_level        rcnn_bl_cls                                                      
leDXgeDX                           2                                                   [0.0, 1.0]   
                                   3            [0.0, 0.6779228627784921, 1.0457047594489426, ...   
                                   4            [0.0, 0.6604345085882057, 1.1252875568613345, ...   
                                   5            [0.0, 0.6686351699358051, 1.0250432923197907, ...   
                                   6            [0.0, 0.6712911516364621, 1.0301814411445482, ...   
...                                                                                           ...   
original    readout_type=inst-last 3                                         [1.6192061943957117]   
                                   4                                         [2.3212554725379357]   
                                   5                                          [3.004800350013854]   
                                   6                                          [3.479933207521204]   
                                   7                                          [3.742125957697331]   

                                                                                        depth_sem  \
data_source aggregate_level        rcnn_bl_cls                                                      
leDXgeDX                           2                                                   [0.0, 0.0]   
                                   3            [0.0, 0.011786384209289003, 0.0022980207696338...   
                                   4            [0.0, 0.009666105934403096, 0.0214346409505206...   
                                   5            [0.0, 0.009182018343403323, 0.0156683274826972...   
                                   6            [0.0, 0.009095930201604553, 0.0171117406484672...   
...                                                                                           ...   
original    readout_type=inst-last 3                                       [0.013629454431964585]   
                                   4                                        [0.03471804278213872]   
                                   5                                       [0.059522291240981406]   
                                   6                                        [0.07704836257130354]   
                                   7                                         [0.1974280159452048]   

                                                  n  \
data_source aggregate_level        rcnn_bl_cls        
leDXgeDX                           2            128   
                                   3            128   
                                   4            128   
                                   5            128   
                                   6            128   
...                                             ...   
original    readout_type=inst-last 3             32   
                                   4             32   
                                   5             32   
                                   6             32   
                                   7             24   

                                                                                        perf_mean  \
data_source aggregate_level        rcnn_bl_cls                                                      
leDXgeDX                           2                     [0.5827205953237654, 0.6362983375148481]   
                                   3            [0.5817766063826616, 0.6336522351918112, 0.645...   
                                   4            [0.5830580295761361, 0.6352792495918914, 0.644...   
                                   5            [0.5828021535659615, 0.6373089954634681, 0.647...   
                                   6            [0.5826850098199584, 0.6373965644740361, 0.648...   
...                                 